# 원핫인코딩 (One-hot Encoding)

In [ ]:
# 앞에서 얘기한 장단점에 맞게 정수 인코딩이랑 원핫인코딩에 맞게 사용하세요 

In [1]:
raw_text = """The Little Prince, written by Antoine de Saint-Exupéry, is a poetic tale about a young prince who travels from his home planet to Earth. The story begins with a pilot stranded in the Sahara Desert after his plane crashes. While trying to fix his plane, he meets a mysterious young boy, the Little Prince.

The Little Prince comes from a small asteroid called B-612, where he lives alone with a rose that he loves deeply. He recounts his journey to the pilot, describing his visits to several other planets. Each planet is inhabited by a different character, such as a king, a vain man, a drunkard, a businessman, a geographer, and a fox. Through these encounters, the Prince learns valuable lessons about love, responsibility, and the nature of adult behavior.

On Earth, the Little Prince meets various creatures, including a fox, who teaches him about relationships and the importance of taming, which means building ties with others. The fox's famous line, "You become responsible, forever, for what you have tamed," resonates with the Prince's feelings for his rose.

Ultimately, the Little Prince realizes that the essence of life is often invisible and can only be seen with the heart. After sharing his wisdom with the pilot, he prepares to return to his asteroid and his beloved rose. The story concludes with the pilot reflecting on the lessons learned from the Little Prince and the enduring impact of their friendship.

The narrative is a beautifully simple yet profound exploration of love, loss, and the importance of seeing beyond the surface of things."""

In [2]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

# 문장 토큰화 
sentences = sent_tokenize(raw_text)

# 영어 불용어 리스트 
from nltk.corpus import stopwords
en_stopwords = stopwords.words('english')  

# 단어사전 (key=단어, value=빈도)
vocab = {}


# 토큰화/정제/정규화 처리 결과 
preprocessed_sentences = []

# 토큰 만큼 반복
for sentence in sentences : 
    sentence = sentence.lower() # 대소문자 정규화 (소문자 변환)

    tokens = word_tokenize(sentence) # 단어 토큰화
    tokens = [token for token in tokens if token not in en_stopwords] # 불용어 제거 
    tokens = [token for token in tokens if len(token) > 2]  # 단어 길이가 2 이하면 제거 

    for token in tokens :
        if token not in vocab:
            vocab[token] = 1
        else :
            vocab[token] += 1
    
    preprocessed_sentences.append(tokens)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = 15, oov_token='<OOV>')
tokenizer.fit_on_texts(preprocessed_sentences)
sequences = tokenizer.texts_to_sequences(preprocessed_sentences)

padded_seqs = pad_sequences(sequences, maxlen=10, truncating='pre')  # 정수 인코딩을 진행한 다음에 원핫인코딩 진행 

In [4]:
from tensorflow.keras.utils import to_categorical

one_hot_encodded = to_categorical(padded_seqs)
one_hot_encodded.shape # (13, 10, 15) : 문장 갯수 13, 토큰 개수 10, 차원 개수 15(num_words 개수 + 1 개의 차원으로 반환함.- to_categorical 작동 방식)

(13, 10, 15)

In [6]:
padded_seqs  # 문장 갯수 13, 토큰 개수 10 

array([[ 1,  1,  1,  1,  7,  2,  1,  1,  8,  9],
       [ 0,  0, 10,  1,  4,  1,  1,  1, 11,  1],
       [ 0,  1,  1, 11, 12,  1,  7,  1,  3,  2],
       [ 1,  1, 13,  1,  1,  1,  1,  5,  1,  1],
       [ 0,  0,  0,  1,  1,  4,  1,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  6],
       [ 1,  2,  1,  1, 14,  1,  1,  1,  1,  1],
       [ 1,  6,  1,  1,  1,  1,  1,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  1,  1,  2,  1,  5],
       [ 1,  3,  2,  1,  1,  1,  1,  1,  1,  1],
       [ 0,  0,  1,  1,  4,  1,  1, 13,  1,  5],
       [ 1,  4,  1, 14,  1,  3,  2,  1,  1,  1],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1]], dtype=int32)

-----

### 한국어 전처리 
1. 토큰화 (형태소 분석)
2. 시퀀스 처리 Tokenizer
3. 패딩 처리 pad_sequences
4. one-hot encoding

In [8]:
texts = [
    "나는 오늘 학원에 간다", 
    "친구들이랑 맛있는 점심 먹을 생각에 신난다.", 
    "오늘은 강사님이 무슨 간식을 줄까?"
]

In [15]:
from konlpy.tag import Okt
import re

okt = Okt()

ko_stopwords = ['은', '는', '이', '가', '을', '를', '와', '과', '에', '의', '으로', '나', '내', '우리', '들']

preprocessed_texts = []

for text in texts :
    tokens = okt.morphs(text, stem=True)  # stem : 기본 형태로 반환하게 해주는 
    tokens = [token for token in tokens if token not in ko_stopwords]  # 불용어 처리 
    tokens = [token for token in tokens if not re.search(r'[\s.,:;?!]', token)] # 공백문자(\), 구두점 제거 
    preprocessed_texts.append(tokens)

In [16]:
preprocessed_texts

[['오늘', '학원', '간다'],
 ['친구', '이랑', '맛있다', '점심', '먹다', '생각', '신나다'],
 ['오늘', '강사', '님', '무슨', '간식', '주다']]

In [ ]:
# 시퀀스 처리 
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(preprocessed_texts)
sequences = tokenizer.texts_to_sequences(preprocessed_texts)  
sequences

[[2, 3, 4], [5, 6, 7, 8, 9, 10, 11], [2, 12, 13, 14, 15, 16]]

In [18]:
tokenizer.word_index

{'<OOV>': 1,
 '오늘': 2,
 '학원': 3,
 '간다': 4,
 '친구': 5,
 '이랑': 6,
 '맛있다': 7,
 '점심': 8,
 '먹다': 9,
 '생각': 10,
 '신나다': 11,
 '강사': 12,
 '님': 13,
 '무슨': 14,
 '간식': 15,
 '주다': 16}

In [19]:
# 패딩 처리 
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_seqs = pad_sequences(sequences, maxlen=3)
padded_seqs

array([[ 2,  3,  4],
       [ 9, 10, 11],
       [14, 15, 16]], dtype=int32)

In [ ]:
# 원핫 인코딩 
from tensorflow.keras.utils import to_categorical

one_hot_encodded = to_categorical(padded_seqs)
# one_hot_encodded
one_hot_encodded.shape # 문장 3개, 3개 자리로 자르기(maxlen=3), 차원 16?

(3, 3, 17)

---

In [ ]:
# 간단한 모델 구조 확인해보기 
from tensorflow.keras import models, layers

input = layers.Input(shape=(3, 17))   # 입력층 정의 - 원핫 인코딩을 한 친구를 넣을거기 때문에 3개의 단어를 17개 차원으로 가지고 있기 떄문에 동일하게 만들었음
x = layers.SimpleRNN(8)(input)        # rnn 모델에 입력 (목요일 설명) - 8개의 노드를 만듦 > 출력층 정의 
output = layers.Dense(1, activation='sigmoid')(x)  # 2진분류를 하기 위해 시그모이드를 활성합수로 정의 

model = models.Model(inputs=input, outputs=output)
model.summary()                                     # 각 레이어에 대한 설명 

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 3, 17)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 8)              │           208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 217 (868.00 B)

 Trainable params: 217 (868.00 B)

 Non-trainable params: 0 (0.00 B)

In [25]:
import numpy as np

model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # 모델에 대한 추가 설정 추가 / 최적화 함수를 adam / 성능평가를 accuracy 사용 
labels = np.array([1, 0, 1])  # 임의의 값으로 라벨 지정 

model.fit(one_hot_encodded, labels, epochs=3)  

Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 756ms/step - accuracy: 0.3333 - loss: 0.9253
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.3333 - loss: 0.9156
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.3333 - loss: 0.9061


반드시 알아야할것 
- 품사 태깅(형태소 이해하기 위해 설명) > 형태소 처리 / 토큰화 처리를 통해 정제와 정규화 진행 (둘의 차이점: 정제는 노이즈 제거, 정규화는 표기 통일(도메인 지식에 맞게 진행))
- 형태를 안맞추면 노이즈 발생 > 어간이나 표제어 추출 (형태 맞추기)
- 정규 표현식 
- 텍스트 데이터 전처리가 완료됐을 때 컴퓨터가 이해하기 쉽게 인코딩 진행
    - 입력 데이터의 길이를 맞추기 위해 진행하는 것이 패딩
    - 어쩌구가 어쩌구 
    - 정수 인코딩 
    - 원핫인코딩
        - 순서에 영향을 덜 받음. 
- 모델에 적용하는 순서를 확인하기 위해 keras를 이용한 모델돌리는 것 진행 